In [15]:
#readme
#this file splits into testing and training

In [11]:
#read in csv file
import csv
#vectorizes data.csv
from sklearn.feature_extraction.text import TfidfVectorizer
#used for dataframes
import pandas as pd
#used for splitting
from sklearn.model_selection import train_test_split
#used to perform SVM
from sklearn import svm
#used for metrics
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
# import graphviz 
#used for upsampling
from sklearn.utils import resample

# Data Frame Creation

In [12]:
#reads in 
df = pd.read_csv('M1.csv', encoding='utf-8')
cat = '7CAT'

In [13]:
df[:5]

,created,original_post,title,url,8CAT,7CAT,6CAT,5CAT,4CAT,org data
0,2015,commun get stuck in sitetosit vpn configur bet...,IPsec VPN site-to-site between ASA and Cisco R...,https://supportforums.cisco.com/discussion/123...,0,1,1,0,1,x
1,2015,everyon problem site site vpn between two rout...,Site to site vpn problem,https://supportforums.cisco.com/discussion/124...,0,1,1,0,1,x
2,2015,enabl alwayson vpn asa 5540 like vpn client ab...,enabling 2 entries in cisco anyconnect client ...,https://supportforums.cisco.com/discussion/124...,0,1,1,0,1,x
3,2015,anyconnect secur mobiliy client give error tri...,Cisco AnyConnect Secure Mobility Client Failed...,https://supportforums.cisco.com/discussion/124...,0,1,1,0,1,x
4,2015,hope someon might abl help stump tri set up an...,Anyconnect on non standard port (eg 444),https://supportforums.cisco.com/discussion/124...,0,1,1,0,1,x


# Creates Training and Testing sets

## Represents with TF-IDF

In [27]:
#Current implementation 
#splits and then runs tf-idf

#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
#creates a training set of 180 values

#used http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
train, test = train_test_split(df, test_size = .1)
#split train set into 
print(len(train)) #601 training posts
print(len(test)) #67

tfidf = TfidfVectorizer(min_df = 2)
#uses vectorizer to perform fit_transform
train_tfs = tfidf.fit_transform(train['original_post'].values.astype('U'))
#transforms to fit the vocabulary of the training set
test_tfs = tfidf.transform(test['original_post'])
whole_tfs = tfidf.fit_transform(df['original_post'])

#creates a list of 180 values correlated to the training set
num_list_train = train['7CAT'].values
#creates a list of 20 values correlated to the test set
num_list_test = test['7CAT'].values
num_list_whole = df['7CAT'].values

601
67


In [28]:
print(df['7CAT'].value_counts())
print(train_tfs.shape)

3    194
6    146
2    107
1     71
5     60
7     47
4     43
Name: 7CAT, dtype: int64
(601, 3169)


# Implements SVM

In [29]:
#sets up instance of svm
support = svm.SVC(C = 1, class_weight = 'balanced',kernel='poly')
#feeds in (matrix, corresponding classificaiton value)
support.fit(train_tfs,num_list_train)

SVC(C=1, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Stratified K-Folds

In [30]:
def metrics():
    """
    STRATIFIED
    Prints out metrics for performace of: 
        SVM Classification
        
    taken from here
    https://machinelearningmastery.com/metrics-evaluate-machine-learning-algorithms-python/
    """
    #creates variables needed for metrics
    #overall_tfs = tfidf.fit_transform(origin['original_post'])
    #predicted = logistic.predict(test_tfs)
    #overall_labels = origin['VPN #']
    
    #stratified cross validation
    skf = StratifiedKFold(n_splits=10)
    skf.get_n_splits(whole_tfs,num_list_whole)
    print(skf)
    
    #for train_index, test_index in skf.split(overall_tfs, overall_labels):
        #print("TRAIN:", train_index, "TEST:", test_index)
        #overall_tfs_train, overall_tfs_test = overall_tfs[train_index], overall_tfs[test_index]
        #overall_labels_train, overall_labels_test = overall_labels[train_index], overall_labels[test_index]

    print("SVM Classification Accuracy:")

    #accuracy
    #results = model_selection.cross_val_score(support, whole_tfs,num_list_whole, cv=skf, scoring='accuracy')
    #print("Accuracy: %.3f (%.3f)\n") % (results.mean(), results.std())

    #confusion matrix
    y_pred = cross_val_predict(support,whole_tfs,num_list_whole,cv=10)
    conf_mat = confusion_matrix(num_list_whole ,y_pred)
    print("Confusion Matrix:\n%s\n"%conf_mat)

    #classification report
    y_pred = cross_val_predict(support,whole_tfs,num_list_whole,cv=10)
    report = classification_report(num_list_whole, y_pred)
    print("Classification Report:\n%s\n"%report)

  
   

In [31]:
#stratified
metrics()

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
SVM Classification Accuracy:
Confusion Matrix:
[[29  0  0 29  0  0 13]
 [41  0  0 50  0  0 16]
 [77  0  0 85  0  0 32]
 [17  0  0 23  0  0  3]
 [24  0  0 24  0  0 12]
 [57  0  0 64  0  0 25]
 [17  0  0 20  0  0 10]]

Classification Report:
             precision    recall  f1-score   support

          1       0.11      0.41      0.17        71
          2       0.00      0.00      0.00       107
          3       0.00      0.00      0.00       194
          4       0.08      0.53      0.14        43
          5       0.00      0.00      0.00        60
          6       0.00      0.00      0.00       146
          7       0.09      0.21      0.13        47

avg / total       0.02      0.09      0.04       668




In [28]:
print(num_list_test)

[3 4 4 4 3 3 2 3 3 3 3 4 3 3 3 3 2 1 3 3]
